In [1]:
%run functions.ipynb

Ready: 2023-04-15 19:51:19


In [19]:
file_name = 'files/kaliningrad/requests_text_kaliningrad_230415_1101.xlsx'
df = pd.read_excel(file_name)
df = df[df['Рейтинг рубрики'] > 0 ].iloc[:15]
rubric = df[df['Рейтинг рубрики'] == 1]['Рубрика'].iloc[0]
city = 'Калининград'
dups = []

df_text = pd.read_excel(file_name, usecols=['Рубрика', 'Рейтинг рубрики', 'Поисковый запрос', 'tx_1'])
df_text = df_text[df_text['tx_1'].notna()]
tx_1 = df_text[df_text['Поисковый запрос'].apply(lambda x: x not in dups)][['tx_1', 'Рейтинг рубрики']]

words = [i for i in tx_1[tx_1['Рейтинг рубрики'] == 1]['tx_1'].values]

In [36]:
def add_image_request(docx_name, image_name):
    try:
        doc = docx.Document(docx_name)
    except:
        doc = docx.Document('files/report.docx')
        doc.save(docx_name)
    image_to_add = doc.add_picture(image_name)
    image_to_add.width = Cm(17.5)
    image_to_add.height = Cm(10.8)
    doc.save(docx_name)

def add_links(docx_name, rubric, words, city):
    try:
        doc = docx.Document(docx_name)
    except:
        doc = docx.Document('files/report.docx')
        doc.save(docx_name)
    p = doc.add_paragraph(rubric + ': ')
    for word in words:
        text = word
        query = word.replace(' ', '+')
        search = f'https://www.google.ru/search?q={query}+{city}'
        add_hyperlink(p, text, search)
        #doc.add_paragraph(', ')
        if word != words[-1]:
            p.add_run(', ')
    doc.save(docx_name)

def add_hyperlink(paragraph, text, url):
    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element and a new w:rPr element
    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    # Create a new Run object and add the hyperlink into it
    r = paragraph.add_run()
    r._r.append (hyperlink)

    # A workaround for the lack of a hyperlink style (doesn't go purple after using the link)
    # Delete this if using a template that has the hyperlink style in it
    #r.font.color.theme_color = MSO_THEME_COLOR_INDEX.HYPERLINK
    #r.font.underline = True

    return hyperlink

def make_images(file_name, city, header_color, row_color):
    dt_now = str(datetime.datetime.now())[2:-9].replace('-','').replace(':','').replace(' ','_')
    df = pd.read_excel(file_name).drop(['tx_1','tx_2','tx_3','tx_4','tx_5'], axis=1)
    word_file = f'{file_name[:-5]}_report_{dt_now}.docx'
    print(word_file)
    
    df.columns = ['Рубрика', 'Поисковый запрос', 'Количество\nзапросов 2020',
           'Количество\nзапросов 2021', 'Количество\nзапросов 2022',
           'Количество\nзапросов 2023', 'Количество\nзапросов, итого',
           'Рейтинг\nпо запросам, итого', 'Рейтинг рубрики']

    for i in df.columns[2:-2]:
        df[i] = df[i].apply(make_sep)
        
    dups = df['Поисковый запрос'][(df['Поисковый запрос'].duplicated())&(df['Рейтинг рубрики'] > 0)].values 
    df_text = pd.read_excel(file_name, usecols=['Рубрика', 'Рейтинг рубрики', 'Поисковый запрос', 'tx_1'])
    df_text = df_text[df_text['tx_1'].notna()]
    tx_1 = df_text[df_text['Поисковый запрос'].apply(lambda x: x not in dups)][['tx_1', 'Рейтинг рубрики']]
    
    for i in tqdm(df['Рейтинг рубрики'].unique()[1:]):  
        dfx = df[df['Рейтинг рубрики'] == i].drop('Рубрика', axis=1).iloc[:20]
        rubric = df[df['Рейтинг рубрики'] == i]['Рубрика'].iloc[0]
        col_name = f'Поисковые запросы по рубрике\n{i:02}. {rubric}'
        img_file_name = file_name.replace('files/','')[:-5]
        img_file_name_final = f'pictures/{img_file_name}_{i:02}.png'
        dfx = dfx.rename({'Поисковый запрос':col_name}, axis=1).drop(['Рейтинг рубрики', 'Рейтинг\nпо запросам, итого'], axis=1)
        words = tx_1[tx_1['Рейтинг рубрики'] == i]['tx_1'].values
        
        render_mpl_table(dfx, img_file_name_final, font_size=19, header_color=header_color, row_colors=[row_color, 'w'], dups=dups)
        add_links(word_file, rubric, words, city)
        add_image_request(word_file, img_file_name_final)

In [ ]:
files/kaliningrad/requests_text_kaliningrad_230415_1101_report_230415_2203.docx

In [ ]:
make_images('files/kaliningrad/requests_text_kaliningrad_230415_1101.xlsx', "Калининград", '#B76C00', '#FFF4D9')